In [ ]:
!pip install tokenizers

     |████████████████████████████████| 6.5 MB 5.1 MB/s 


# New Section

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

from tokenizers import BertWordPieceTokenizer
import numpy as np

In [ ]:
import tokenizers

In [ ]:
print(hub.__version__)

0.12.0


In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'
bert_layer = hub.KerasLayer(tfhub_handle_encoder,trainable=True)

In [ ]:
import time
t1=time.time()
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'
bert_layer = hub.KerasLayer(tfhub_handle_encoder,trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy().decode("utf-8")
tokenizer = BertWordPieceTokenizer(vocab=vocab_file)
print(time.time()-t1)

38.43530631065369


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Sample:
    def __init__(self, question, context, start_char_idx=None, answer_text=None, answer_exist=1):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.answer_exist = answer_exist
        self.skip = False
        self.start_token_idx = -1
        self.end_token_idx = -1

    def preprocess(self):
        # tokenize context and question
        tokenized_context = tokenizer.encode(self.context)
        tokenized_question = tokenizer.encode(self.question)
        
        # if this is validation or training sample, preprocess answer
        if self.answer_text is not None:
            # check if end character index is in the context
            end_char_idx = self.start_char_idx + len(self.answer_text)
            if end_char_idx >= len(self.context):
                self.skip = True
                return
        
            # mark all the character indexes in context that are also in answer     
            is_char_in_ans = [0] * len(self.context)
            for idx in range(self.start_char_idx, end_char_idx):
                is_char_in_ans[idx] = 1
            ans_token_idx = []
        
            # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context.offsets):
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
        
            if len(ans_token_idx) == 0:
                self.skip = True
                return
        
            # get start and end token indexes
            if self.answer_exist == 1:
                self.start_token_idx = ans_token_idx[0]
                self.end_token_idx = ans_token_idx[-1]
            else:
                self.start_token_idx = 0
                self.end_token_idx = 0
                
        # create inputs as usual
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            self.skip = True
            return
        
        self.input_word_ids = input_ids
        self.input_type_ids = token_type_ids
        self.input_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'
model_name = 'bert-chaii'
model_name = 'bert-chaii'
max_seq_length = 512

word_sep = ' '
max_word_len = 128
word_overlap = 64

In [ ]:
def create_bert_inputs(samples):
    dataset_dict = {
        "input_word_ids": [],
        "input_type_ids": [],
        "input_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }

    for item in samples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))

    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [dataset_dict["input_word_ids"],
         dataset_dict["input_mask"],
         dataset_dict["input_type_ids"]]

    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    
    return x, y

In [ ]:
def make_spans(question, context, answer_start=None, answer_text=None):
    span_texts, words = make_span_texts(context)
    spans = []
    total_char_len = 0
    span_no = 0

    for s in span_texts:
        span_context = word_sep.join(s)
        span_char_len = len(span_context)
        span_answer_start = 0
        span_answer_exist = 0
        
        if answer_text is not None:
            if (answer_start >= total_char_len and  answer_start < (total_char_len + span_char_len)): # Answer exist
                span_answer_exist = 1
                if span_no > 0:
                    span_answer_start = answer_start - total_char_len
                else:
                    span_answer_start =  answer_start # for span 0
            
        span = [question, span_context, span_answer_start, answer_text, span_answer_exist]
        spans.append(span)
        
        total_char_len = total_char_len + span_char_len
        span_no = span_no + 1

    return spans

In [ ]:
def make_span_texts(text):
    words = text.split(word_sep)
    nwords = len(words)
    remainder = nwords % max_word_len
    nspans = int((nwords - remainder) / max_word_len) + 1
    spans = []

    for x in range(nspans):
        start = x*(max_word_len-word_overlap)
        end = start + max_word_len
        if(end > nwords): end = nwords
        span = words[start:end]
        spans.append(span)

    return spans, words

In [ ]:
def chaii_test_data(ques, text):
    samples = []
    question = ques
    context = text
    spans = make_spans(question, context)
    for s in spans:
        s = Sample(s[0], s[1])
        s.preprocess()
        samples.append(s)
            
    return samples

In [ ]:
!unzip /content/drive/MyDrive/bert-chaii.zip

Archive:  /content/drive/MyDrive/bert-chaii.zip
   creating: bert-chaii/assets/
  inflating: bert-chaii/assets/vocab.txt  
  inflating: bert-chaii/keras_metadata.pb  
  inflating: bert-chaii/saved_model.pb  
   creating: bert-chaii/variables/
  inflating: bert-chaii/variables/variables.data-00000-of-00001  
  inflating: bert-chaii/variables/variables.index  


In [ ]:
model = tf.keras.models.load_model(model_name)

In [ ]:
def testModel(ques,text):
    
    #model = tf.keras.models.load_model(model_name)
    test_samples = chaii_test_data(ques,text)
    test_samples = [x for x in test_samples if x.skip == False]
    xt, _ = create_bert_inputs(test_samples)
    
    pred_start, pred_end = model.predict(xt)
    
    for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
        
        test_sample = test_samples[idx]
        offsets = test_sample.context_token_to_char
        start = np.argmax(start)
        end = np.argmax(end)
        pred_ans = None
        
        if start >= end : continue
        if start >= len(offsets): continue

        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_ans = test_sample.context[pred_char_start:offsets[end][1]]
        else:
            pred_ans = test_sample.context[pred_char_start:]
        #print("here in test model")
        #print("Q: " + test_sample.question)
        #print("A: " + pred_ans)
        return pred_ans

Context:    
सीऐटल (अंग्रेजी: Seattle) अमेरिका के वाशिंगटन राज्य का एक प्रमुख शहर है। यह वाशिंगटन राज्य का सबसे बड़ा शहर होने के साथ-साथ वहाँ का प्रमुख बन्दरगाह भी है। यह प्रशान्त महासागर तथा लेक वौशिन्ग्टन के बीच स्थित है। कनाडा की सीमा यहाँ से केवल १६० किलोमीटर दूर है। अप्रैल २००९ में यहाँ की आबादी लगभग ६१७०० थी।पाइक प्लेस मार्केट यहाँ की बड़ी मशहूर सब्जी मंडी है। पर्य्टक एवं निवासी रोज फल, सब्जियाँ, फूल, मछली आदी ख्ररीदनें यहाँ हजारों की तादाद् में आते हैं।मानव यहाँ कम-से-कम ४००० र्वषों से बसा हुआ है। गोरों का आगमन सन १८५१ में शुरु हुआ। आर्थ्रर डेन्नी तथा उनके साथियों ने सबसे पह्ली बस्ती बसायी जिसका नाम न्यू यॉर्क-ऍल्काइ रखा गया। सन १८५३ में दुवामिश तथा सुवामिश कबीलों के सरदार सिआलह को सम्मानित करने के लिये बस्ती का नाम सिऐटल रखा गया।श्रेणी:अमेरिका के शहर

Question:     
सीटल शहर कहाँ स्थित है?

In [ ]:
testModel("सीटल शहर कहाँ स्थित है?","सीऐटल (अंग्रेजी: Seattle) अमेरिका के वाशिंगटन राज्य का एक प्रमुख शहर है। यह वाशिंगटन राज्य का सबसे बड़ा शहर होने के साथ-साथ वहाँ का प्रमुख बन्दरगाह भी है। यह प्रशान्त महासागर तथा लेक वौशिन्ग्टन के बीच स्थित है। कनाडा की सीमा यहाँ से केवल १६० किलोमीटर दूर है। अप्रैल २००९ में यहाँ की आबादी लगभग ६१७०० थी।पाइक प्लेस मार्केट यहाँ की बड़ी मशहूर सब्जी मंडी है। पर्य्टक एवं निवासी रोज फल, सब्जियाँ, फूल, मछली आदी ख्ररीदनें यहाँ हजारों की तादाद् में आते हैं।मानव यहाँ कम-से-कम ४००० र्वषों से बसा हुआ है। गोरों का आगमन सन १८५१ में शुरु हुआ। आर्थ्रर डेन्नी तथा उनके साथियों ने सबसे पह्ली बस्ती बसायी जिसका नाम न्यू यॉर्क-ऍल्काइ रखा गया। सन १८५३ में दुवामिश तथा सुवामिश कबीलों के सरदार सिआलह को सम्मानित करने के लिये बस्ती का नाम सिऐटल रखा गया।श्रेणी:अमेरिका के शहर")

Q: सीटल शहर कहाँ स्थित है?
A: प्रशान्त महासागर तथा लेक वौशिन्ग्टन के बीच


Context:    
पैटन मैनिंग दो अलग-अलग टीमों को मल्टीपल सुपर बाउल में लीड लड़ने वाले पहले क्वार्टरबैक बने। वह 39 में उम्र में एक सुपर बाउल खेलने वाले सबसे अधिक उम्र के क्वार्टरबैक भी हैं। पिछला रिकॉर्ड जॉन एलवे के नाम था, जिन्होंने 38 की उम्र में सुपर बाउल XXXIII जीतने में ब्रोंकोस का नेतृत्व किया और वर्तमान में डेनवर के फुटबॉल संचालन के कार्यकारी उपाध्यक्ष और महाप्रबंधक हैं।

Question:     
सुपर बाउल में खेलने वाले सबसे अधिक उम्र के क्वार्टरबैक होने का रिकॉर्ड पहले किसके पास था?

In [ ]:
testModel("सुपर बाउल में खेलने वाले सबसे अधिक उम्र के क्वार्टरबैक होने का रिकॉर्ड पहले किसके पास था?","पैटन मैनिंग दो अलग-अलग टीमों को मल्टीपल सुपर बाउल में लीड लड़ने वाले पहले क्वार्टरबैक बने। वह 39 में उम्र में एक सुपर बाउल खेलने वाले सबसे अधिक उम्र के क्वार्टरबैक भी हैं। पिछला रिकॉर्ड जॉन एलवे के नाम था, जिन्होंने 38 की उम्र में सुपर बाउल XXXIII जीतने में ब्रोंकोस का नेतृत्व किया और वर्तमान में डेनवर के फुटबॉल संचालन के कार्यकारी उपाध्यक्ष और महाप्रबंधक हैं।")

Q: सुपर बाउल में खेलने वाले सबसे अधिक उम्र के क्वार्टरबैक होने का रिकॉर्ड पहले किसके पास था?
A: पैटन मैनिंग


Context:    
जैसा कि हाउस ऑफ कॉमन्स में, कई योग्यताएं MSP होने के लिए लागू होती हैं। ऐसी योग्यताएं हाउस ऑफ कॉमन्स डिसक्वालिफिकेशन एक्ट 1975 और ब्रिटिश नेशनलिटी एक्ट 1981 के तहत पेश की गई थी। विशेष रूप से, सदस्यों को 18 से अधिक उम्र का होना चाहिए और यूनाइटेड किंगडम, आयरलैंड गणराज्य, राष्ट्रमंडल देशों में से एक देश, ब्रिटिश विदेशी क्षेत्र का नागरिक, या ब्रिटेन में एक यूरोपीय संघ का निवासी होना चाहिए। पुलिस और सशस्त्र बलों के सदस्य को स्कॉटिश संसद में निर्वाचित MSP के रूप में बैठने से अयोग्य ठहराया जाता है, और इसी तरह, सिविल सेवकों और विदेशी विधानसभाओं के सदस्यों को अयोग्य घोषित किया जाता है। एक व्यक्ति स्कॉटिश संसद में नहीं बैठ सकता है यदि उसे मानसिक स्वास्थ्य (देखभाल और उपचार) (स्कॉटलैंड) अधिनियम 2003 की शर्तों के तहत पागल माना जाता है।

Question:      
हाउस ऑफ कॉमन्स के साथ MSP का हिस्सा क्या है?

In [ ]:
testModel("हाउस ऑफ कॉमन्स के साथ MSP का हिस्सा क्या है?","जैसा कि हाउस ऑफ कॉमन्स में, कई योग्यताएं MSP होने के लिए लागू होती हैं। ऐसी योग्यताएं हाउस ऑफ कॉमन्स डिसक्वालिफिकेशन एक्ट 1975 और ब्रिटिश नेशनलिटी एक्ट 1981 के तहत पेश की गई थी। विशेष रूप से, सदस्यों को 18 से अधिक उम्र का होना चाहिए और यूनाइटेड किंगडम, आयरलैंड गणराज्य, राष्ट्रमंडल देशों में से एक देश, ब्रिटिश विदेशी क्षेत्र का नागरिक, या ब्रिटेन में एक यूरोपीय संघ का निवासी होना चाहिए। पुलिस और सशस्त्र बलों के सदस्य को स्कॉटिश संसद में निर्वाचित MSP के रूप में बैठने से अयोग्य ठहराया जाता है, और इसी तरह, सिविल सेवकों और विदेशी विधानसभाओं के सदस्यों को अयोग्य घोषित किया जाता है। एक व्यक्ति स्कॉटिश संसद में नहीं बैठ सकता है यदि उसे मानसिक स्वास्थ्य (देखभाल और उपचार) (स्कॉटलैंड) अधिनियम 2003 की शर्तों के तहत पागल माना जाता है।")

Q: हाउस ऑफ कॉमन्स के साथ MSP का हिस्सा क्या है?
A: कई योग्यताए


Context:   
उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र कहां है जिसके लिए तनाव-टेंसर की गणना की जा रही है। इस नियम-निष्ठता में दबाव की शर्तें शामिल होती हैं जो क्रॉस-सेक्शनल क्षेत्र (टैंसर के मैट्रिक्स विकर्ण) के साथ-साथ कतरनी शर्तों के साथ सामान्य रूप से कार्य करती हैं और जो क्रॉस-सेक्शनल क्षेत्र (ऑफ-डायमेंशनल एलिमेंट्स) के समानांतर कार्य करती हैं। तनाव टेंसर बलों के कारण होता है जो सभी तनावों (विकृतियों) का कारण बनता है जिसमें तन्य तनाव और संपीडन भी शामिल हैं ।:133–134:38-1–38-11

Question:    
आयतन में क्षेत्र की गणना करते समय दबाव की क्या शर्तें शामिल हैं?

In [ ]:
testModel("आयतन में क्षेत्र की गणना करते समय दबाव की क्या शर्तें शामिल हैं?","उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र कहां है जिसके लिए तनाव-टेंसर की गणना की जा रही है। इस नियम-निष्ठता में दबाव की शर्तें शामिल होती हैं जो क्रॉस-सेक्शनल क्षेत्र (टैंसर के मैट्रिक्स विकर्ण) के साथ-साथ कतरनी शर्तों के साथ सामान्य रूप से कार्य करती हैं और जो क्रॉस-सेक्शनल क्षेत्र (ऑफ-डायमेंशनल एलिमेंट्स) के समानांतर कार्य करती हैं। तनाव टेंसर बलों के कारण होता है जो सभी तनावों (विकृतियों) का कारण बनता है जिसमें तन्य तनाव और संपीडन भी शामिल हैं ।:133–134:38-1–38-11")

Q: आयतन में क्षेत्र की गणना करते समय दबाव की क्या शर्तें शामिल हैं?
A: नियम-निष्ठता


Context:   
उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र कहां है जिसके लिए तनाव-टेंसर की गणना की जा रही है। इस नियम-निष्ठता में दबाव की शर्तें शामिल होती हैं जो क्रॉस-सेक्शनल क्षेत्र (टैंसर के मैट्रिक्स विकर्ण) के साथ-साथ कतरनी शर्तों के साथ सामान्य रूप से कार्य करती हैं और जो क्रॉस-सेक्शनल क्षेत्र (ऑफ-डायमेंशनल एलिमेंट्स) के समानांतर कार्य करती हैं। तनाव टेंसर बलों के कारण होता है जो सभी तनावों (विकृतियों) का कारण बनता है जिसमें तन्य तनाव और संपीडन भी शामिल हैं ।:133–134:38-1–38-11

Question:   
सामान्य ताकतों से क्या जुड़ा है?

In [ ]:
testModel("सामान्य ताकतों से क्या जुड़ा है?","उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र कहां है जिसके लिए तनाव-टेंसर की गणना की जा रही है। इस नियम-निष्ठता में दबाव की शर्तें शामिल होती हैं जो क्रॉस-सेक्शनल क्षेत्र (टैंसर के मैट्रिक्स विकर्ण) के साथ-साथ कतरनी शर्तों के साथ सामान्य रूप से कार्य करती हैं और जो क्रॉस-सेक्शनल क्षेत्र (ऑफ-डायमेंशनल एलिमेंट्स) के समानांतर कार्य करती हैं। तनाव टेंसर बलों के कारण होता है जो सभी तनावों (विकृतियों) का कारण बनता है जिसमें तन्य तनाव और संपीडन भी शामिल हैं ।:133–134:38-1–38-11")

Q: सामान्य ताकतों से क्या जुड़ा है?
A: दबाव की शर्त


Context:   
पूंजीवाद, अभिजात वर्ग और साम्राज्यवाद के बीच संबंध लंबे समय से इतिहासकारों और राजनीतिक सिद्धांतकारों के बीच बहस के मुद्दे रहे हैं। इस बहस के अधिकांश हिस्से के अगुआ जे. ए. होबसन (1858-1940), जोसेफ शम्पेटर (1883-1950), थोरस्टीन वेबलिन (1857-1929) और नॉर्मन एंगेल (1872-1967) जैसे सिद्धांतकार थे। जबकि ये गैर-मार्क्सवादी लेखक प्रथम विश्व युद्ध से पहले सबसे अधिक सफल थे, वे युद्ध काल के वर्षों में सक्रिय रहे। उनके संयुक्त कार्य ने साम्राज्यवाद के अध्ययन और इसके यूरोप पर प्रभाव की जानकारी दी, साथ ही साथ संयुक्त राज्य अमेरिका में 1950 के दशक से सैन्य-राजनीतिक संकुल के उदय की भावना में योगदान दिया। हॉबसन ने तर्क दिया कि घरेलू सामाजिक सुधार साम्राज्यवाद की अंतरराष्ट्रीय बीमारी को उसकी आर्थिक नींव को हटाकर ठीक कर सकते हैं। हॉब्सन ने कहा कि कराधान के माध्यम से राज्य का हस्तक्षेप व्यापक खपत को बढ़ावा दे सकता है, धन का सृजन कर सकता है और एक शांतिपूर्ण, सहिष्णु, बहुध्रुवीय विश्व व्यवस्था को प्रोत्साहित कर सकता है।

Question:   
कुछ बहस के अनुसार पूंजीवाद, साम्राज्यवाद, और किस चीज़ के बीच एक संबंध है?

In [ ]:
testModel("कुछ बहस के अनुसार पूंजीवाद, साम्राज्यवाद, और किस चीज़ के बीच एक संबंध है?","पूंजीवाद, अभिजात वर्ग और साम्राज्यवाद के बीच संबंध लंबे समय से इतिहासकारों और राजनीतिक सिद्धांतकारों के बीच बहस के मुद्दे रहे हैं। इस बहस के अधिकांश हिस्से के अगुआ जे. ए. होबसन (1858-1940), जोसेफ शम्पेटर (1883-1950), थोरस्टीन वेबलिन (1857-1929) और नॉर्मन एंगेल (1872-1967) जैसे सिद्धांतकार थे। जबकि ये गैर-मार्क्सवादी लेखक प्रथम विश्व युद्ध से पहले सबसे अधिक सफल थे, वे युद्ध काल के वर्षों में सक्रिय रहे। उनके संयुक्त कार्य ने साम्राज्यवाद के अध्ययन और इसके यूरोप पर प्रभाव की जानकारी दी, साथ ही साथ संयुक्त राज्य अमेरिका में 1950 के दशक से सैन्य-राजनीतिक संकुल के उदय की भावना में योगदान दिया। हॉबसन ने तर्क दिया कि घरेलू सामाजिक सुधार साम्राज्यवाद की अंतरराष्ट्रीय बीमारी को उसकी आर्थिक नींव को हटाकर ठीक कर सकते हैं। हॉब्सन ने कहा कि कराधान के माध्यम से राज्य का हस्तक्षेप व्यापक खपत को बढ़ावा दे सकता है, धन का सृजन कर सकता है और एक शांतिपूर्ण, सहिष्णु, बहुध्रुवीय विश्व व्यवस्था को प्रोत्साहित कर सकता है।")

here in test model
Q: कुछ बहस के अनुसार पूंजीवाद, साम्राज्यवाद, और किस चीज़ के बीच एक संबंध है?
A: अभिजात वर्ग


Context:   
8 फरवरी 2007 को, BSkyB ने अपने तीन फ्री-टू-एयर डिजिटल टेरेस्ट्रियल चैनलों को चार सब्सक्रिप्शन चैनल्स के साथ बदलने के अपने विचार की घोषणा की। यह प्रस्तावित किया गया कि ये चैनल BSkyB पोर्टफोलियो से खेल (इंग्लिश प्रीमियर लीग फुटबॉल सहित), फिल्मों, मनोरंजन और समाचार सहित कई ऑफर प्रस्तुत करेंगे। घोषणा के एक दिन बाद सेतांता स्पोर्ट्स ने पुष्टि कि यह मार्च में डिजिटल टेरेस्ट्रियल प्लेटफॉर्म पर एक सब्सक्रिप्शन सेवा के रूप में लॉन्च किया जायेगा, और उसी दिन एनटीएल की सेवाओं को वर्जिन मीडिया का नाम दिया गया। हालांकि, उद्योग के सूत्रों का मानना है कि संभावित खोए हुए विज्ञापन राजस्व के कारण, BSkyB को अपने चैनल को फ्रीव्यू से हटाने और सब्सक्रिप्शन चैनलों को बदलने की योजना बनाने पर मजबूर होना पड़ेगा।

Question:  
सेतांता स्पोर्ट्स ने एक सब्सक्रिप्शन सेवा के रूप में लॉन्च करने के लिए कब कहा ?

In [ ]:
testModel("सेतांता स्पोर्ट्स ने एक सब्सक्रिप्शन सेवा के रूप में लॉन्च करने के लिए कब कहा ?","8 फरवरी 2007 को, BSkyB ने अपने तीन फ्री-टू-एयर डिजिटल टेरेस्ट्रियल चैनलों को चार सब्सक्रिप्शन चैनल्स के साथ बदलने के अपने विचार की घोषणा की। यह प्रस्तावित किया गया कि ये चैनल BSkyB पोर्टफोलियो से खेल (इंग्लिश प्रीमियर लीग फुटबॉल सहित), फिल्मों, मनोरंजन और समाचार सहित कई ऑफर प्रस्तुत करेंगे। घोषणा के एक दिन बाद सेतांता स्पोर्ट्स ने पुष्टि कि यह मार्च में डिजिटल टेरेस्ट्रियल प्लेटफॉर्म पर एक सब्सक्रिप्शन सेवा के रूप में लॉन्च किया जायेगा, और उसी दिन एनटीएल की सेवाओं को वर्जिन मीडिया का नाम दिया गया। हालांकि, उद्योग के सूत्रों का मानना है कि संभावित खोए हुए विज्ञापन राजस्व के कारण, BSkyB को अपने चैनल को फ्रीव्यू से हटाने और सब्सक्रिप्शन चैनलों को बदलने की योजना बनाने पर मजबूर होना पड़ेगा।")

Q: सेतांता स्पोर्ट्स ने एक सब्सक्रिप्शन सेवा के रूप में लॉन्च करने के लिए कब कहा ?
A: मार्च


Context:  
एक कागज का टुकड़ा बाद में पाया गया जिसपर लूथर ने अपनाआखिरी कथन लिखा। यह कथन “हम याचक हैं” जो जर्मन में था,  के अतिरिक्त लैटिन में था।

Question:   
लूथर के द्वारा लिखा गया बाद में क्या खोजा गया?



In [ ]:
testModel("लूथर के द्वारा लिखा गया बाद में क्या खोजा गया?","एक कागज का टुकड़ा बाद में पाया गया जिसपर लूथर ने अपनाआखिरी कथन लिखा। यह कथन “हम याचक हैं” जो जर्मन में था,  के अतिरिक्त लैटिन में था।")

Q: लूथर के द्वारा लिखा गया बाद में क्या खोजा गया?
A: अपनाआखिरी कथन


In [ ]:
testModel("लूथर के द्वारा लिखा गया बाद में क्या खोजा गया?","एक कागज का टुकड़ा बाद में पाया गया जिसपर लूथर ने अपनाआखिरी कथन लिखा। यह कथन “हम याचक हैं” जो जर्मन में था,  के अतिरिक्त लैटिन में था।")

Q: लूथर के द्वारा लिखा गया बाद में क्या खोजा गया?
A: अपनाआखिरी कथन
